# Data Cleaning

## Import Packages

In [1]:
import pandas as pd
import numpy as np
from itertools import chain
from functools import reduce

## Get Country Codes

In [2]:
country_df = pd.read_excel(
    '../data/raw/unwto-all-data-download_2022.xlsx',
    sheet_name = 'Inbound Tourism-Arrivals',
    header = 2
)

In [3]:
country_df = country_df[['C.','Basic data and indicators']][
    (country_df['Basic data and indicators'].isna() == False) & 
    (country_df['C.'].isna() == False)
]

In [4]:
countries = country_df.rename(columns = {
    'C.':'country_code',
    'Basic data and indicators':'country'
})

## Load Country Mapping File

In [5]:
country_mapping_df = pd.read_csv('../data/processed/country_mapping.csv')

## Function to Standardize Data Cleaning for Tourism Sheets

In [6]:
def cleanse_tourism_df(sheet):
    
    # read excel sheet
    df = pd.read_excel(
        '../data/raw/unwto-all-data-download_2022.xlsx',
        sheet_name = sheet,
        header = 2
    )
    
    # remove irrelevant rows
    df = df[
        (df['S.'].isna() == False) & 
        (df['S.'] != 0)
    ]
    
    # utilizing the attribute number column with table name, as numbers are repeating across tables
    df['table_measure'] = sheet.replace(' ','_').replace('-','_').lower() + '_' + df['S.'].map('{:,.2f}'.format)
    
    # defining the columns to keep
    col_list = [['C.','table_measure'], list(range(1995,2023))]
    col_list = list(chain(*col_list))
    df = df[col_list]
    
    # utilizing pd.melt() to put the years onto rows
    df = pd.melt(df,
                 id_vars=['C.', 'table_measure'], 
                 var_name='year',
                 value_name='value')
    
    # replacing '..' values with NaN
    df['value'] = df['value'].replace('..',np.nan)
    
    # pivoting to put the attributes onto columns
    df = df.pivot(index=['C.', 'year'], columns='table_measure', values='value')#.reset_index()
    
    return df

## Creating Data Frames

In [7]:
# inbound tourism tables
inbound_accommodations = cleanse_tourism_df('Inbound Tourism-Accommodation')
inbound_regions = cleanse_tourism_df('Inbound Tourism-Regions')
inbound_arrivals = cleanse_tourism_df('Inbound Tourism-Arrivals')
inbound_purpose = cleanse_tourism_df('Inbound Tourism-Purpose')
inbound_transport = cleanse_tourism_df('Inbound Tourism-Transport')
inbound_expenditure = cleanse_tourism_df('Inbound Tourism-Expenditure')

# domestic tourism tables
domestic_trips = cleanse_tourism_df('Domestic Tourism-Trips')
domestic_accommodations = cleanse_tourism_df('Domestic Tourism-Accommodation')

# outbound tourism tables
outbound_departures = cleanse_tourism_df('Outbound Tourism-Departures')
outbound_expenditure = cleanse_tourism_df('Outbound Tourism-Expenditure')

# tourism industry table
tourism_industry = cleanse_tourism_df('Tourism Industries')

# tourism employment table
employment = cleanse_tourism_df('Employment')

/var/folders/n7/35x1v29d0bzbkcsdc706mjxw0000gn/T/ipykernel_28521/1419121283.py:31: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['value'] = df['value'].replace('..',np.nan)
/var/folders/n7/35x1v29d0bzbkcsdc706mjxw0000gn/T/ipykernel_28521/1419121283.py:31: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['value'] = df['value'].replace('..',np.nan)
/var/folders/n7/35x1v29d0bzbkcsdc706mjxw0000gn/T/ipykernel_28521/1419121283.py:31: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future versio

In [8]:
# concatenate data frames into one
tourism_df = pd.concat([
    inbound_accommodations,
    inbound_regions,
    inbound_arrivals,
    inbound_purpose,
    inbound_transport,
    inbound_expenditure,
    domestic_trips,
    domestic_accommodations,
    outbound_departures,
    outbound_expenditure,
    tourism_industry,
    employment
], axis = 1)

tourism_df = tourism_df.reset_index().rename(columns = {
    'C.':'country_code'
})

In [9]:
column_name_map = {
    'country_code':'country_code',
    'year':'year',
    'inbound_tourism_accommodation_1.29':'total_guests',
    'inbound_tourism_accommodation_1.30':'total_overnights',
    'inbound_tourism_accommodation_1.31':'hotel_guests',
    'inbound_tourism_accommodation_1.32':'hotel_overnights',
    'inbound_tourism_regions_1.10':'total_middle_east',
    'inbound_tourism_regions_1.11':'total_south_asia',
    'inbound_tourism_regions_1.12':'total_other_region_not_classified',
    'inbound_tourism_regions_1.13':'total_nationals_residing_abroad',
    'inbound_tourism_regions_1.50':'total_regions',
    'inbound_tourism_regions_1.60':'total_africa',
    'inbound_tourism_regions_1.70':'total_americas',
    'inbound_tourism_regions_1.80':'total_east_asia_pacific',
    'inbound_tourism_regions_1.90':'total_europe',
    'inbound_tourism_arrivals_1.10':'total_region_arrivals', #remove
    'inbound_tourism_arrivals_1.20':'overnight_visitors',
    'inbound_tourism_arrivals_1.30':'same_day_visitors',
    'inbound_tourism_arrivals_1.40':'same_day_cruise_passengers',
    'inbound_tourism_purpose_1.14':'total_by_purpose',
    'inbound_tourism_purpose_1.15':'visitors_personal',
    'inbound_tourism_purpose_1.18':'visitors_business_professional',
    'inbound_tourism_transport_1.19':'total_arrivals_transport', # remove
    'inbound_tourism_transport_1.20':'arrivals_by_air',
    'inbound_tourism_transport_1.21':'arrivals_by_water',
    'inbound_tourism_transport_1.22':'arrivals_by_land',
    'inbound_tourism_expenditure_1.33':'total_inbound_tourism_expenditure',
    'inbound_tourism_expenditure_1.34':'travel_inbound_tourism_expenditure',
    'inbound_tourism_expenditure_1.35':'transport_inbound_tourism_expenditure',
    'domestic_tourism_trips_2.10':'total_domestic_trip_visitors',
    'domestic_tourism_trips_2.20':'domestic_trip_overnight_visitors',
    'domestic_tourism_trips_2.30':'domestic_trip_same_day_visitors',
    'domestic_tourism_accommodation_2.19':'total_guests_domestic',
    'domestic_tourism_accommodation_2.20':'total_overnights_domestic',
    'domestic_tourism_accommodation_2.21':'hotel_guests_domestic',
    'domestic_tourism_accommodation_2.22':'hotel_overnights_domestic',
    'outbound_tourism_departures_3.10':'total_departures',
    'outbound_tourism_departures_3.20':'departures_overnight_visitors',
    'outbound_tourism_departures_3.30':'departures_same_day_visitors',
    'outbound_tourism_expenditure_3.40':'outbound_total_expenditure',
    'outbound_tourism_expenditure_3.50':'outbound_travel_expenditure',
    'outbound_tourism_expenditure_3.60':'outbound_transport_expenditure',
    'tourism_industries_4.13':'establishments',
    'tourism_industries_4.14':'rooms',
    'tourism_industries_4.15':'bed_places',
    'tourism_industries_4.16':'occupancy_rate_per_room',
    'tourism_industries_4.17':'occupancy_rate_per_bed_place',
    'tourism_industries_4.18':'avg_stay_length',
    'tourism_industries_4.19':'available_capacity',
    'employment_5.10':'total_tourism_employees',
    'employment_5.20':'employees_visitor_accomodation_services',
    'employment_5.30':'employees_other_accomodation_services',
    'employment_5.40':'employees_food_and_beverage_service',
    'employment_5.50':'employees_passenger_transportation',
    'employment_5.60':'employees_travel_agencies',
    'employment_5.70':'employees_other_tourism_industries'
}

In [10]:
tourism_df = tourism_df.rename(columns = column_name_map)

## Cleaning Economic Datasets

### GDP Per Capita

In [11]:
# read in excel file
gdp_per_capita_total = pd.read_excel(
    '../data/raw/RealPerCapitaGDPValues.xlsx',
    sheet_name = 'GDP Per Cap',
    header = 12
).rename(columns = {
    'Country':'country'
})

# get columns of interest
gdp_per_capita_cols = [x for x in gdp_per_capita_total.columns if (x.startswith('y')) or (x == 'country')]

# subset to columns, and use pd.melt to get years into a single column
gdp_per_capita = pd.melt(gdp_per_capita_total[gdp_per_capita_cols],
        id_vars=['country'],
        var_name='year',
        value_name='gdp_per_capita')

# replace the 'y' in the year values
gdp_per_capita.year = gdp_per_capita.year.str.replace('y','')

### GDP

In [12]:
# read in excel file
gdp_total = pd.read_excel(
    '../data/raw/RealGDPValues.xlsx',
    sheet_name = 'Real GDP',
    header = 12
).rename(columns = {
    'Country':'country'
})

# get columns of interest
gdp_cols = [x for x in gdp_total.columns if (x.startswith('y')) or (x == 'country')]

# subset to columns, and use pd.melt to get years into a single column
gdp = pd.melt(gdp_total[gdp_cols],
        id_vars=['country'],
        var_name='year',
        value_name='gdp')

# replace the 'y' in the year values
gdp.year = gdp.year.str.replace('y','')

### Exchange Rates

In [14]:
# read in excel file
exchange_rate_total = pd.read_excel(
    '../data/raw/RealExchangeRateValues.xlsx',
    sheet_name = 'Real XR',
    header = 10
).rename(columns = {
    'Country':'country'
})

# get columns of interest
#exchange_rate_cols = exchange_rate_total.columns
exchange_rate_cols = [x for x in exchange_rate_total.columns if (x.startswith('y')) or (x == 'country')]
exchange_rate_cols2 = [x for x in exchange_rate_total.columns if (x.startswith('g')) or (x == 'country')]

# subset to columns, and use pd.melt to get years into a single column
exchange_rates1 = pd.melt(exchange_rate_total[exchange_rate_cols],
        id_vars=['country'],
        var_name='year',
        value_name='exchange_rate')

exchange_rates2 = pd.melt(exchange_rate_total[exchange_rate_cols2],
        id_vars=['country'],
        var_name='year',
        value_name='growth_rate')

# replace the 'y' in the year values
exchange_rates1.year = exchange_rates1.year.str.replace('y','')
exchange_rates2.year = exchange_rates2.year.str.replace('g','')
exchange_rates = pd.merge(exchange_rates1, exchange_rates2, how = 'outer', on = ['year', 'country'])

### Consumer Price Index

In [15]:
# read in excel file
cpi_total = pd.read_excel(
    '../data/raw/CPIValues.xlsx',
    sheet_name = 'CPI',
    header = 11
).rename(columns = {
    'Country':'country'
})

# get columns of interest
cpi_cols = [x for x in cpi_total.columns if (x.startswith('y')) or (x == 'country')]

# subset to columns, and use pd.melt to get years into a single column
cpi = pd.melt(cpi_total[cpi_cols],
        id_vars=['country'],
        var_name='year',
        value_name='cpi')

# replace the 'y' in the year values
cpi.year = cpi.year.str.replace('y','')

### Combine Economic Data

In [16]:
# create list of data frames
econ_datasets = [gdp_per_capita, gdp, exchange_rates, cpi]

# merge all the data frames using reduce
econ_df_merged = reduce(lambda left, right: pd.merge(left, right, on = ['country','year'], how = 'outer'), econ_datasets)

# convert year column to integer
econ_df_merged.year = econ_df_merged.year.astype(int)

# get list of countries (non-aggregated rows)
econ_countries = list(country_mapping_df.econ_dataset_country.dropna())

# filter to country rows, and between 1990 and 2023
econ_df = econ_df_merged.query("(country in @econ_countries) and (year >= 1990) and (year <= 2022)").sort_values(by = ['country','year'], ascending = [True,True])

## Human Development Index Processing

In [17]:
# read csv file
hdi = pd.read_csv('../data/raw/HDR23-24_Composite_indices_complete_time_series (1).csv')

drop_cols = [col for col in hdi.columns if (col.startswith('hdi_m')) or (col.startswith('hdi_f'))] + ['hdi_rank_2022']

# drop hdi rank column
hdi = hdi.drop(columns = drop_cols)

# select columns of interest
hdi = hdi[['country', 'hdicode'] + [col for col in hdi.columns if col.startswith(('hdi_'))]].rename(
    columns = {'hdicode':'hdi_code'}
)

# use pd.melt to put years into one column
hdi = hdi.melt(id_vars=['country', 'hdi_code'], var_name='year',value_name='hdi')

# remove 'hdi' in year values
hdi['year'] = hdi['year'].str.replace(r'hdi_', '', regex=True)

# get list of countries (non-aggregated rows)
hdi_countries = list(country_mapping_df.hdi_dataset_country.dropna())

# filter to countries
hdi = hdi.query("country in @hdi_countries")

## Merging All Data

### Mapping to Common Country Names

In [18]:
# create a dictionary for mapping tourism data frame countries
tourism_country_map = dict(zip(country_mapping_df.tourism_dataset_country, country_mapping_df.country_value_map))

# get the country values from country codes
tourism_mapped = pd.merge(tourism_df, countries)

# convert year to int
tourism_mapped.year = tourism_mapped.year.astype(int)

# map country values
tourism_mapped.country = tourism_mapped.country.replace(tourism_country_map)

In [19]:
# create a dictionary for mapping economics data frame countries
econ_map_df = country_mapping_df.query("econ_dataset_country.isna() == False")
econ_country_map = dict(zip(econ_map_df.econ_dataset_country, econ_map_df.country_value_map))

# take a copy of the economics data frame
econ_df_mapped = econ_df.copy()

# convert year to int
econ_df_mapped.year = econ_df_mapped.year.astype(int)

# map country values
econ_df_mapped.country = econ_df_mapped.country.replace(econ_country_map)

In [20]:
# create a dictionary for mapping hdi data frame countries
hdi_map_df = country_mapping_df.query("hdi_dataset_country.isna() == False")
hdi_country_map = dict(zip(hdi_map_df.hdi_dataset_country, hdi_map_df.country_value_map))

# take a copy of the hdi data frame
hdi_mapped = hdi.copy()

# convert year to int
hdi_mapped.year = hdi_mapped.year.astype(int)

# map country values
hdi_mapped.country = hdi_mapped.country.replace(hdi_country_map)

### Merging

In [21]:
# create list of data frames to merge
df_list = [tourism_mapped, econ_df_mapped, hdi_mapped]

# merge data frames using reduce
df_merged = reduce(lambda left, right: pd.merge(left, right, on = ['country','year'], how = 'left'), df_list)

In [22]:
# getting region
df_merged = pd.merge(df_merged, country_mapping_df, left_on = 'country', right_on = 'country_value_map').rename(
    columns = {'region_value_map':'world_region'}
)

In [23]:
# select relevant columns
final_df = df_merged[[
       'world_region', 'country', 'year', 'total_guests', 'total_overnights',
       'hotel_guests', 'hotel_overnights', 'total_middle_east',
       'total_south_asia', 'total_other_region_not_classified',
       'total_nationals_residing_abroad', 'total_regions', 'total_africa',
       'total_americas', 'total_east_asia_pacific', 'total_europe',
       'total_region_arrivals', 'overnight_visitors', 'same_day_visitors',
       'same_day_cruise_passengers', 'total_by_purpose', 'visitors_personal',
       'visitors_business_professional', 'total_arrivals_transport',
       'arrivals_by_air', 'arrivals_by_water', 'arrivals_by_land',
       'total_inbound_tourism_expenditure',
       'travel_inbound_tourism_expenditure',
       'transport_inbound_tourism_expenditure', 'total_domestic_trip_visitors',
       'domestic_trip_overnight_visitors', 'domestic_trip_same_day_visitors',
       'total_guests_domestic', 'total_overnights_domestic',
       'hotel_guests_domestic', 'hotel_overnights_domestic',
       'total_departures', 'departures_overnight_visitors',
       'departures_same_day_visitors', 'outbound_total_expenditure',
       'outbound_travel_expenditure', 'outbound_transport_expenditure',
       'establishments', 'rooms', 'bed_places', 'occupancy_rate_per_room',
       'occupancy_rate_per_bed_place', 'avg_stay_length', 'available_capacity',
       'total_tourism_employees', 'employees_visitor_accomodation_services',
       'employees_other_accomodation_services',
       'employees_food_and_beverage_service',
       'employees_passenger_transportation', 'employees_travel_agencies',
       'employees_other_tourism_industries', 'gdp_per_capita',
       'gdp', 'exchange_rate', 'cpi', 'hdi_code', 'hdi'
]]

## Exporting Data to CSV

In [24]:
# write to csv
tourism_df.to_csv('../data/processed/tourism_metrics.csv', index = False)
countries.to_csv('../data/processed/country_codes.csv', index = False)
gdp_per_capita.to_csv('../data/processed/gdp_per_capita.csv', index = False)
gdp.to_csv('../data/processed/gdp.csv', index=False)
exchange_rates.to_csv('../data/processed/exchange_rates.csv', index=False)
cpi.to_csv('../data/processed/consumer_price_index.csv', index=False)
hdi.to_csv('../data/processed/hdi.csv', index=False)
econ_df.to_csv('../data/processed/economic_metrics.csv', index=False)
final_df.to_csv('../data/processed/full_tourism_dataset.csv', index=False)